# ***REVENUE INSIGHTS IN HOSPITALITY DOMAIN***

1. Problem Statement: Led the analysis of revenue streams in the hospitality industry. Provided actionable insights on occupancy rates, revenue per available room(RevPAR), and seasonal trends. Helped the business improve pricing strategies and maximize revenue growth.

2. Gathering of data: Collect relevant data as per the business problem. (Here I gathered data from 'CodeBasics'.)

3. Importing libraries and load the datasets:

In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_date = pd.read_csv('/content/dim_date.csv')
df_hotels = pd.read_csv('/content/dim_hotels.csv')
df_rooms = pd.read_csv('/content/dim_rooms.csv')
df_aggbook = pd.read_csv('/content/fact_aggregated_bookings.csv')
df_book = pd.read_csv('/content/fact_bookings.csv')

In [77]:
#print(type(df_date))

4. Data Cleaning and Transformation:


In [78]:
df_date.head()

,date,mmm yy,week no,day_type
0,01-May-22,May 22,W 19,weekend
1,02-May-22,May 22,W 19,weekeday
2,03-May-22,May 22,W 19,weekeday
3,04-May-22,May 22,W 19,weekeday
4,05-May-22,May 22,W 19,weekeday


In [79]:
# Deleting the 'day_type' column because in this domain friday and saturday are considered as weekend.
# Later we calculate and make separate columns.

df_date = df_date.drop(columns=['day_type'])
df_date.head()

,date,mmm yy,week no
0,01-May-22,May 22,W 19
1,02-May-22,May 22,W 19
2,03-May-22,May 22,W 19
3,04-May-22,May 22,W 19
4,05-May-22,May 22,W 19


In [80]:
df_date['wn'] = df_date['week no'].str.extract(r'(\d+)')
df_date['wn'] = pd.to_numeric(df_date['wn'])
df_date.head(10)

,date,mmm yy,week no,wn
0,01-May-22,May 22,W 19,19
1,02-May-22,May 22,W 19,19
2,03-May-22,May 22,W 19,19
3,04-May-22,May 22,W 19,19
4,05-May-22,May 22,W 19,19
5,06-May-22,May 22,W 19,19
6,07-May-22,May 22,W 19,19
7,08-May-22,May 22,W 20,20
8,09-May-22,May 22,W 20,20
9,10-May-22,May 22,W 20,20


In [81]:
from datetime import datetime

df_date['date'] = pd.to_datetime(df_date['date'], format='%d-%b-%y')

df_date['weekday'] = df_date['date'].dt.day_name()

df_date['day_type'] = df_date['date'].dt.weekday

for x in df_date.index:
  if df_date.loc[x, 'day_type'] == 4 or df_date.loc[x, 'day_type'] == 5:
    df_date.loc[x, 'day_type'] = 'Weekend'
  else:
    df_date.loc[x, 'day_type'] = 'Weekday'
df_date.head(10)

<ipython-input-81-0e48926549b2>:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Weekday' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_date.loc[x, 'day_type'] = 'Weekday'


,date,mmm yy,week no,wn,weekday,day_type
0,2022-05-01,May 22,W 19,19,Sunday,Weekday
1,2022-05-02,May 22,W 19,19,Monday,Weekday
2,2022-05-03,May 22,W 19,19,Tuesday,Weekday
3,2022-05-04,May 22,W 19,19,Wednesday,Weekday
4,2022-05-05,May 22,W 19,19,Thursday,Weekday
5,2022-05-06,May 22,W 19,19,Friday,Weekend
6,2022-05-07,May 22,W 19,19,Saturday,Weekend
7,2022-05-08,May 22,W 20,20,Sunday,Weekday
8,2022-05-09,May 22,W 20,20,Monday,Weekday
9,2022-05-10,May 22,W 20,20,Tuesday,Weekday


In [82]:
df_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      92 non-null     datetime64[ns]
 1   mmm yy    92 non-null     object        
 2   week no   92 non-null     object        
 3   wn        92 non-null     int64         
 4   weekday   92 non-null     object        
 5   day_type  92 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 4.4+ KB


In [83]:
df_date.describe()

,date,wn
count,92,92.000000
mean,2022-06-15 12:00:00,25.076087
min,2022-05-01 00:00:00,19.000000
25%,2022-05-23 18:00:00,22.000000
50%,2022-06-15 12:00:00,25.000000
75%,2022-07-08 06:00:00,28.000000
max,2022-07-31 00:00:00,32.000000
std,NaN,3.812166


In [84]:
df_hotels.head()
df_hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   property_id    25 non-null     int64 
 1   property_name  25 non-null     object
 2   category       25 non-null     object
 3   city           25 non-null     object
dtypes: int64(1), object(3)
memory usage: 928.0+ bytes


In [85]:
df_rooms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   room_id     4 non-null      object
 1   room_class  4 non-null      object
dtypes: object(2)
memory usage: 192.0+ bytes


In [86]:
df_aggbook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   property_id          9200 non-null   int64 
 1   check_in_date        9200 non-null   object
 2   room_category        9200 non-null   object
 3   successful_bookings  9200 non-null   int64 
 4   capacity             9200 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 359.5+ KB


In [87]:
df_book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134590 entries, 0 to 134589
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   booking_id         134590 non-null  object 
 1   property_id        134590 non-null  int64  
 2   booking_date       134590 non-null  object 
 3   check_in_date      134590 non-null  object 
 4   checkout_date      134590 non-null  object 
 5   no_guests          134590 non-null  int64  
 6   room_category      134590 non-null  object 
 7   booking_platform   134590 non-null  object 
 8   ratings_given      56683 non-null   float64
 9   booking_status     134590 non-null  object 
 10  revenue_generated  134590 non-null  int64  
 11  revenue_realized   134590 non-null  int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 12.3+ MB


In [88]:
df_date.describe(include=['object'])

,mmm yy,week no,weekday,day_type
count,92,92,92,92
unique,3,14,7,2
top,May 22,W 19,Sunday,Weekday
freq,31,7,14,66


In [89]:
df_hotels.describe(include=['object'])

,property_name,category,city
count,25,25,25
unique,7,2,4
top,Atliq Grands,Luxury,Mumbai
freq,4,16,8


In [90]:
df_rooms.describe(include=['object'])

,room_id,room_class
count,4,4
unique,4,4
top,RT1,Standard
freq,1,1


In [91]:
df_aggbook.describe(include=['object'])

,check_in_date,room_category
count,9200,9200
unique,92,4
top,01-May-22,RT1
freq,100,2300


In [92]:
df_book.describe(include=['object'])

,booking_id,booking_date,check_in_date,checkout_date,room_category,booking_platform,booking_status
count,134590,134590,134590,134590,134590,134590,134590
unique,134590,116,92,97,4,7,3
top,May012216558RT11,2022-06-08,2022-07-16,2022-05-09,RT2,others,Checked Out
freq,1,1670,2017,1840,49505,55066,94411


In [94]:
cleaned_file_path = '/content/dim_date_cleaned.csv'
x = df_date.to_csv(cleaned_file_path, index=False)